# Ecuaciones diferenciales parciales
Las ecuaciones diferenciales parciales (EDPs) constituyen una área de suma importancia en la física, ya que modelan sistemas que varían con respecto a más de una variable independiente, por ejemplo, tanto el tiempo como el espacio.

Del punto de vista numérico, el tipo de EDPs que es (conceptualmente) más sencillo son las llamadas **parabólicas**, es decir, **ecuaciones de evolución**, de las cuales la más conocida es la **ecuación de calor** o **ecuación de difusión**.

# Ecuación de calor
La ecuación de calor modela el esparcimiento en el tiempo y en el espacio de un "paquete" de calor (perturbación local de temperatura en una región) o un "paquete" de concentración de una sustancia física o química. 

Llamemos $u(t, \mathbf{x})$ la temperatura o la concentración de la sustancia en la posición $\mathbf{x}$ al tiempo $t$. Recordemos que la ecuación de calor es

$$\frac{\partial u(t, \mathbf{x})}{\partial t} = D \, \nabla^2 u(t, \mathbf{x}),$$

con $\nabla^2 := \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}$ en tres dimensiones.

Esta ecuación nos dice cómo varía la concentración en el tiempo, dadas las condiciones locales en el espacio. Se deriva en términos de una ley de conservación:

$$\frac{\partial u}{\partial t} + \nabla \cdot \mathbf{J} = 0,$$

donde el flujo de calor o de concentración $\mathbf{J}$ es proporcional a la gradiente local:

$$\mathbf{J} = -D \, \nabla u.$$

La ecuación de calor es una **ecuación de evolución** que describe cómo evoluciona el sistema en el tiempo. 

## Una dimensión

Empecemos con el caso más sencillo: con sólo una dimensión espacial. En este caso, la ecuación de calor se reduce a

$$\frac{\partial u(t, x)}{\partial t} = D \frac{\partial^2 u(t, x)}{\partial x^2}.$$

Para resolverla, necesitaremos además:
- una condición inicial $u(t=0, x) = f(x)$ (una función del espacio)
- condiciones en la frontera, $u(t, x)$ para todo $x$ en la frontera del dominio espacial, y para todo $t$.

## Métodos numéricos para la ecuación de calor
Dado que, como siempre, no podemos resolver problemas de naturaleza continua en la computadora, debemos *aproximar* la solución $u(t, x)$ de alguna forma. La manera más sencilla es utilizar una **discretización**.

Para discretizar la EDP, aproximamos $$\frac{\partial u}{\partial t} (t_n, x_i) \approx \frac{u_i ^{n+1} - u_i ^n}{h}$$

Por otro lado, $$u(t,x+h) + u(t,x-h) \approx 2u(t,x) + h^2 \ddot{u}(t,x) + \mathcal{O}(h^3)$$ 
Por lo que $$\frac{\partial^2 u}{\partial x^2} (t_n,x_i) \approx \frac{u(t_n, x_i + k) - 2u(t_n,x_i) + u(t_n, x_i - k)}{k^2} \approx \frac{u_{i+1}^n -2u_i ^n + u_{î-1}^n}{k^2}$$

Así la ecuacion discretizada queda $$\frac{u_i ^{n+1} - u_i ^n}{h} = D\frac{u_{i+1}^n - 2u_i ^n + u_{i-1}^n}{k^2}$$

Entonces se obtiene: $$u_i ^{n+1} = u_i ^n + D\frac{h}{k^2}(u_{i+1}^n - 2u_i ^n + u_{i-1}^n)$$

In [2]:
Pkg.add("OffsetArrays")

INFO: Cloning cache of OffsetArrays from https://github.com/JuliaArrays/OffsetArrays.jl.git
INFO: Installing OffsetArrays v0.3.0
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of OffsetArrays
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [3]:
using OffsetArrays #Usamos esto para para poder utilizar arreglos cuyos índices empiecen en -L y terminen en L

INFO: Precompiling module OffsetArrays.


El siguiente método es la discretización de la ecuación de calor para una dimensión en un intervalo de $x=-L$ a $x=L$, con condición inicial $u(t=0, x) = \delta(x)$, donde $\delta$ es la delta de Dirac, y condiciones de frontera absorbentes (de Dirichlet), es decir, $u(t, -L) = u(t, +L) = 0$ para todo $t > 0$.

Para representar a la función delta de Dirac, se puede promediar la función $$u_0 (x_i) = \frac{\int_{x_i - \tfrac{k}{2}}^{x_i + \tfrac{k}{2}} f(x) dx}{k}$$
Si $f(x) = \delta(x-x_i)$ entonces da $\tfrac{1}{k}$ en $x_i$ y 0 $\forall   j \neq i$

In [19]:
function calor_delta(h,k,t0,tf,L,D) #Con esto hacemos más general la función, pues podemos elegir las condiciones iniciales en t 
    th = Float64[]
    xk = Float64[]
    for i in t0:h:tf
        push!(th,i)
    end
    for i in -L:k:L #en una dimensión estamos considerando el intervalo [-L,L]
        push!(xk,i)
    end
    U = Float64[]
    for i in 1:length(xk)
        if xk[i] == 0
             push!(U,1/k)
        else
            push!(U,0.0)
        end
    end
    Ui = [transpose(U) ; zeros((length(th) -1),length(xk))] #Se usa transpose para transponer la matriz
    for i in 2:length(th)
        for j in 1:length(U)
            if j == 1
                Ui[i,j] = 0.0
            elseif j == length(U)
                Ui[i,j] = 0.0
            else
                Ui[i,j] = Ui[i - 1,j] + D*(h/(k^2))*(Ui[i - 1, j + 1] - 2*Ui[i - 1,j] + Ui[i - 1, j - 1])
            end
        end
    end
    return Ui
end

calor_delta (generic function with 1 method)

In [20]:
calor_delta(0.1,0.1,0,5,5,0.01) #Ejemplo

51×101 Array{Float64,2}:
 0.0  0.0      0.0        0.0          …  0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0          …  0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0          …  0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 ⋮                                     ⋱                      ⋮  
 0.0  0.0      0.0        0.0             0.0      

## Dos dimensiones

Recordando que $\nabla^2 u(x,y,t) = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}$, entonces podemos discretizar la ecuación de la siguiente forma:

Aproximando $$\frac{\partial u}{\partial t} (t_n, x_i, y_j) \approx \frac{u_{i,j} ^{n+1} - u_{i,j} ^n}{h}$$

Por otro lado, $$u(t,x + k,y + k) + u(t,x - k, y - k) \approx 2u(t,x,y) + k^2 \ddot{u}(t,x,y) + \mathcal{O}(k^3)$$

Donde $\ddot{u}(t,x,y) = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}$,
por lo que $$\left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\right)(t_n,x_i) \approx \frac{u(t_n, x_i + k,y_j) - 2u(t_n,x_i,y_j) + u(t_n, x_i - k, y_j)}{k^2} + \frac{u(t_n, x_i,y_j + k) - 2u(t_n,x_i,y_j) + u(t_n, x_i, y_j - k)}{k^2}\approx \frac{u_{i+1,j}^n -2u_{i,j} ^n + u_{i-1,j}^n}{k^2} + \frac{u_{i,j + 1}^n -2u_{i,j} ^n + u_{i,j - 1}^n}{k^2}$$

Por lo que la ecuacion discretizada queda $$\frac{u_{i,j} ^{n+1} - u_{i,j} ^n}{h} = D\frac{u_{i+1,j}^n - 2u_{i,j} ^n + u_{i-1,j}^n + u_{i,j + 1}^n -2u_{i,j} ^n + u_{i,j - 1}^n}{k^2}$$

Así, se obtiene: $$u_{i,j} ^{n+1} = u_{i,j} ^n + D\frac{h}{k^2}(u_{i+1,j}^n + u_{i,j + 1}^n - 2(u_{i,j} ^n + u_{i,j} ^n) + u_{i-1,j}^n + u_{i,j - 1}^n)$$

Ahora, las condiciones de frontera de Dirichlet son $u(t, -L, -L) = u(t, +L, +L) = 0,  \forall t > 0$.

De estas condiciones, se espera que la difusión se vaya esparciendo a lo largo de la región de estudio, pero que la "concentración" sea muy baja en las fronteras de la región.

A continuación se muestra el método para la función de calor con condicones de Dirichlet:

In [17]:
function calor_delta2D(h,k,t0,tf,L,D) #Definimos el método como con la ecuación en 1D
    th = Float64[]
    xk = Float64[]
    yk = Float64[]
    for i in t0:h:tf
        push!(th,i)
    end
    for i in -L:k:L 
        push!(xk,i)
    end
    U = Float64[]
    for i in 1:length(xk)
        if xk[i] == 0
             push!(U,1/k)
        else
            push!(U,0.0)
        end
    end  
    Ui = [transpose(U) ; zeros((length(th) -1),length(xk))]
    for i in 2:length(th)
        for j in 1:length(U)
            if j == 1
                Ui[i,j] = 0.0
            elseif j == length(U)
                Ui[i,j] = 0.0
            else
                Ui[i,j] = Ui[i - 1,j] + D*(h/(k^2))*(Ui[i - 1, j + 1] + Ui[i - 1,j] - 2*(Ui[i - 1,j] + Ui[i - 1,j]) + Ui[i - 1,                           j - 1] + Ui[i - 1,j])
            end
        end
    end
    return Ui
end

calor_delta2D (generic function with 1 method)

In [18]:
calor_delta2D(0.1,0.1,0,10,10,0.01) #Ejemplo

101×201 Array{Float64,2}:
 0.0  0.0      0.0        0.0          …  0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0          …  0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0          …  0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 0.0  0.0      0.0        0.0             0.0        0.0      0.0
 ⋮                                     ⋱                      ⋮  
 0.0  0.0      0.0        0.0             0.0     

#### Referencias:
__[1]ZEMANSKY, Mark.Calor y termódinámica. McGraw Hill. Sexta edición.__

__[2]"Discretización" en http://materias.fi.uba.ar/7609/material/Clase%2002/03%20a%20Sistemas%20Discretos.pdf. Revisado: 04/NOV/2017 19:00__